# Arbitrage calculations sandbox

In [1]:
from middleware_app.src.utilities.Bet import Bet

To find arbitrage opportunities we need the following:
 - access to live betting odds from multiple bookmakers
 For each event in the bookmakers database:
 - calculate the implied probability of the event outcome
 - determine if an arb exists for the event using $\left(\frac{1}{OddsX_a}\right) + \left(\frac{1}{OddsY_b}\right) < 1$
 - calculate a balanced betting amount using 
``` Arb_Profit_Percentage = (1 / ((1 / OddsX_a) + (1 / OddsY_b))) - 1
Total_Bet_Amount = Desired_Profit / Arb_Profit_Percentage
Bet_On_A = Total_Bet_Amount * (1 / OddsX_a)
Bet_On_B = Total_Bet_Amount * (1 / OddsY_b)
```
We can use a couple of methods to determine our total bet amount. The simplest form would be a fixed stake, ie. 
 - Conservative: 1% to 2% of your bankroll per bet.
 - Moderate: 3% to 5% of your bankroll per bet.
 - Aggressive: More than 5% of your bankroll per bet (not generally recommended).

A more involved strategy would be to use the kelly criterion
$$ K_{\%} = ((Odds_{dec}) (Implied Probability - 1)) / (Odds_{dec} - 1) $$

Let's just say for now that we'll use a fixed percentage of our bankroll for every bet

Steps:
 - set up a method to generate betting odds, if we have many bookmakers then we could store them like this 
 
```
events = {
    'event_1': {
        'outcome_1': {'bookmaker_1': odds_1_1_1, 'bookmaker_2': odds_1_1_2, ...},
        'outcome_2': {'bookmaker_1': odds_1_2_1, 'bookmaker_2': odds_1_2_2, ...},
        ...
    },
    ...
}
```

Heres some sample data to work with, there are arbitrage opportunities on event 1, 3, and 5.  

In [11]:
events = {
    "Football Match - Team A vs Team B": {
        "Team A Wins": {
            "Bookmaker1": 2.10,
            "Bookmaker2": 2.00,
            "Bookmaker3": 2.05
        },
        "Team B Wins": {
            "Bookmaker1": 1.80,
            "Bookmaker2": 1.95,
            "Bookmaker3": 1.90
        },
        "Draw": {
            "Bookmaker1": 3.50,
            "Bookmaker2": 3.40,
            "Bookmaker3": 3.45
        }
    },
    "Tennis Match - Player X vs Player Y": {
        "Player X Wins": {
            "Bookmaker1": 1.30,
            "Bookmaker2": 1.35,
            "Bookmaker3": 1.33
        },
        "Player Y Wins": {
            "Bookmaker1": 3.60,
            "Bookmaker2": 3.50,  # Arbitrage opportunity here!
            "Bookmaker3": 3.55
        }
    },
    "Basketball Game - Team C vs Team D": {
        "Team C Wins": {
            "Bookmaker1": 1.60,
            "Bookmaker2": 1.65,
            "Bookmaker3": 1.63
        },
        "Team D Wins": {
            "Bookmaker1": 2.40,
            "Bookmaker2": 2.45,  # Arbitrage opportunity here!
            "Bookmaker3": 2.35
        }
    }
}

In [14]:
def check_arbitrage_opportunities(events):
    arbitrage_opportunities = []

    for event, outcomes in events.items():
        best_odds_per_outcome = [max(bookmakers_odds.values()) for outcome_name, bookmakers_odds in outcomes.items()]
        inverse_odds_sum = sum(1/odd for odd in best_odds_per_outcome)

        if inverse_odds_sum < 1:
            print('Arbitrage opportunity found!')
            arbitrage_opportunity = {
                'event': event,
                'best_odds_per_outcome': best_odds_per_outcome,
                'inverse_odds_sum': inverse_odds_sum
            }
            arbitrage_opportunities.append(arbitrage_opportunity)

    return arbitrage_opportunities

# Check for arbitrage opportunities in the 'events_with_arbitrage'
arbitrage_found = check_arbitrage_opportunities(events_with_arbitrage)

for opportunity in arbitrage_found:
    print(f"Arbitrage opportunity found in {opportunity['event']}")


In [21]:
events_with_arbitrage = {
    'Tennis Match: Player A vs Player B': {
        'Player A Wins': {
            'Bookmaker1': 2.40,
            'Bookmaker2': 2.00,
            'Bookmaker3': 1.95
        },
        'Player B Wins': {
            'Bookmaker1': 1.75,
            'Bookmaker2': 1.80,
            'Bookmaker3': 1.70
        }
    },
    'Soccer Game: Team X vs Team Y': {
        'Team X Wins': {
            'Bookmaker1': 2.50,
            'Bookmaker2': 2.55,
            'Bookmaker3': 2.45
        },
        'Draw': {
            'Bookmaker1': 3.30,
            'Bookmaker2': 3.25,
            'Bookmaker3': 3.35
        },
        'Team Y Wins': {
            'Bookmaker1': 2.90,
            'Bookmaker2': 2.85,
            'Bookmaker3': 2.80
        }
    },
}


def check_arbitrage_opportunities(events):
    arbitrage_opportunities = []

    for event, outcomes in events.items():
        print(f'event: {event}')
        print(f'outcomes  : {outcomes}')
        best_odds_per_outcome = [max(bookmakers_odds.values()) for outcome_name, bookmakers_odds in outcomes.items()]
        print(best_odds_per_outcome)
        inverse_odds_sum = sum(1/odd for odd in best_odds_per_outcome)
        print(inverse_odds_sum)
        if inverse_odds_sum < 1:
            print('Arbitrage opportunity found!')
            arbitrage_opportunity = {
                'event': event,
                'best_odds_per_outcome': best_odds_per_outcome,
                'inverse_odds_sum': inverse_odds_sum
            }
            arbitrage_opportunities.append(arbitrage_opportunity)

    return arbitrage_opportunities


# Now, let's use this sample data with the adjusted function.
arbitrage_found = check_arbitrage_opportunities(events_with_arbitrage)

for opportunity in arbitrage_found:
    print(f"Arbitrage opportunity found in {opportunity['event']}")


[2.4, 1.8]
0.9722222222222223
Arbitrage opportunity found!
[2.55, 3.35, 2.9]
1.035491911638562
Arbitrage opportunity found in Tennis Match: Player A vs Player B


Let's create a json file of these odds for testing

In [131]:
import random
import json 

def generate_similar_odds(base_odds, variance=0.1):
    return base_odds * random.uniform(1 - variance, 1 + variance)

def create_events(num_events, num_books, base_odds_a, base_odds_b, variance=0.15):
    events = {}
    for i in range(1, num_events + 1):
        # Base odds modified to be closer together
        base_odds_a, base_odds_b = sorted([
            random.uniform(base_odds_a, base_odds_b),
            random.uniform(base_odds_a, base_odds_b)
        ])
        
        event_name = f'event_{i}'
        events[event_name] = {
            'a': {f'Bookmaker{j+1}': generate_similar_odds(base_odds_a, variance) for j in range(num_books)},
            'b': {f'Bookmaker{j+1}': generate_similar_odds(base_odds_b, variance) for j in range(num_books)}
        }
    return events

# Settings: Change these values as needed
num_events = 100 
num_books = 6
variance = 0.0000075 # Increased variance
base_odds_a = 2.1  # Base odds for outcome 'a'
base_odds_b = 1.9  # Base odds for outcome 'b'

# Generate the events with similar odds
events_with_similar_odds = create_events(num_events, num_books, base_odds_a, base_odds_b, variance)

# Rest of your script...


# Print the generated event odds
for event_name, outcomes in events_with_similar_odds.items():
    print(event_name)
    for outcome, bookmakers in outcomes.items():
        print(f"  Outcome '{outcome}':", bookmakers)


# Write to a JSON file
with open('arbitrage_calc\data\odds_data.json', 'w') as outfile:
    json.dump(events_with_similar_odds, outfile)

print(f"Generated JSON with {num_events} events and odds from {num_books} bookmakers.")


event_1
  Outcome 'a': {'Bookmaker1': 1.9080328393005892, 'Bookmaker2': 1.908023804668208, 'Bookmaker3': 1.9080433044850287, 'Bookmaker4': 1.9080393802687339, 'Bookmaker5': 1.9080236295701607, 'Bookmaker6': 1.908032654145943}
  Outcome 'b': {'Bookmaker1': 2.0242030360342556, 'Bookmaker2': 2.0242032088942357, 'Bookmaker3': 2.0241943493228276, 'Bookmaker4': 2.0241910316813656, 'Bookmaker5': 2.02421297733712, 'Bookmaker6': 2.0242007394794284}
event_2
  Outcome 'a': {'Bookmaker1': 1.9256174178248042, 'Bookmaker2': 1.9256190360947296, 'Bookmaker3': 1.9256318694251753, 'Bookmaker4': 1.9256247095746781, 'Bookmaker5': 1.925608772464417, 'Bookmaker6': 1.925620174635141}
  Outcome 'b': {'Bookmaker1': 1.929908181100173, 'Bookmaker2': 1.9299060320086863, 'Bookmaker3': 1.9299017714805, 'Bookmaker4': 1.929893649386092, 'Bookmaker5': 1.9298859883046253, 'Bookmaker6': 1.9299035188442561}
event_3
  Outcome 'a': {'Bookmaker1': 1.9257731230247523, 'Bookmaker2': 1.9257727343962623, 'Bookmaker3': 1.9257658

In [132]:
import json
from concurrent.futures import ThreadPoolExecutor

# Function to load events data from a JSON file
def load_events_from_json(file_path):
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

def check_for_arbitrage(event_name, outcomes):
    best_odds_per_outcome = [max(bookmakers_odds.values()) for outcome_name, bookmakers_odds in outcomes.items()]
    inverse_odds_sum = sum(1 / odd for odd in best_odds_per_outcome)
    
    if inverse_odds_sum < 1:
        return {
            'event': event_name,
            'best_odds_per_outcome': best_odds_per_outcome,
            'inverse_odds_sum': inverse_odds_sum
        }
    else:
        return None

def find_arbitrage_opportunities(events):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_for_arbitrage, event, outcomes) for event, outcomes in events.items()]
        
        arbitrage_opportunities = []
        for future in futures:
            result = future.result()
            if result is not None:
                arbitrage_opportunities.append(result)
                
    return arbitrage_opportunities

# Read the odds data from the JSON file
events_with_arbitrage = load_events_from_json('arbitrage_calc/data/odds_data.json')

# Now we use the new parallel function to find arbitrage opportunities
arbitrage_found = find_arbitrage_opportunities(events_with_arbitrage)

for opportunity in arbitrage_found:
    print(f"Arbitrage opportunity found in {opportunity['event']}")


Let's mock this up as if it's coming from a bookies API and build the async logic

In [93]:
from concurrent.futures import ThreadPoolExecutor

events_with_arbitrage = {
    'Tennis Match: Player A vs Player B': {
        'Player A Wins': {
            'Bookmaker1': 2.40,
            'Bookmaker2': 2.00,
            'Bookmaker3': 1.95
        },
        'Player B Wins': {
            'Bookmaker1': 1.75,
            'Bookmaker2': 1.80,
            'Bookmaker3': 1.70
        }
    },
    'Soccer Game: Team X vs Team Y': {
        'Team X Wins': {
            'Bookmaker1': 2.50,
            'Bookmaker2': 2.55,
            'Bookmaker3': 2.45
        },
        'Draw': {
            'Bookmaker1': 3.30,
            'Bookmaker2': 3.25,
            'Bookmaker3': 3.35
        },
        'Team Y Wins': {
            'Bookmaker1': 2.90,
            'Bookmaker2': 2.85,
            'Bookmaker3': 2.80
        }
    },
}

def check_for_arbitrage(event_name, outcomes):
    best_odds_per_outcome = [max(bookmakers_odds.values()) for outcome_name, bookmakers_odds in outcomes.items()]
    inverse_odds_sum = sum(1 / odd for odd in best_odds_per_outcome)
    
    if inverse_odds_sum < 1:
        return {
            'event': event_name,
            'best_odds_per_outcome': best_odds_per_outcome,
            'inverse_odds_sum': inverse_odds_sum
        }
    else:
        return None

def find_arbitrage_opportunities(events):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_for_arbitrage, event, outcomes) for event, outcomes in events.items()]
        
        arbitrage_opportunities = []
        for future in futures:
            result = future.result()
            if result is not None:
                arbitrage_opportunities.append(result)
                
    return arbitrage_opportunities

# Now we use the new parallel function to find arbitrage opportunities
arbitrage_found = find_arbitrage_opportunities(events_with_arbitrage)

for opportunity in arbitrage_found:
    print(f"Arbitrage opportunity found in {opportunity['event']}")


Arbitrage opportunity found in Tennis Match: Player A vs Player B
